## Introduction

- talk about the objective of this assignment 
- opentron
- measure response surface
- talk about statistical design of experiements what it is / how it works

## Analysis

### Set up environment and assignment folder

Before we begin, let's set up our environment and import the necessary libraries.

If we are using an API key, we need to set uo the environment variable for the API key. 
- Create a .env text file in the root directory of your project and save the key as MPI_KEY=your_api_key_here
- In a Jupyter cell run the following:



In [ ]:
"""
import os
from dotenv import load_dotenv
load_dotenv()
MPI_KEY = os.getenv("MPI_KEY")
"""

Create a virtual environment in the terminal 
- python -m venv .venv  

Create a new text file with the name ".gitignore"
- add the text venv/,pycache/ and .env (if used)

**Issues arrived from multiple python versions that kept conflicting with each other  
Before beginning ensure that 3.13.9 and pymatgen 2025.10.7 are running**


In [ ]:
import sys
import pkg_resources

print("Python version:", sys.version)
print("pymatgen version:", pkg_resources.get_distribution("pymatgen").version)

Check everything is in order:
- make sure this is the main repository on the local drive
    - pwd
- make sure this is the main repository url
    - git remote -v
- we need to add our new files from the assignment folder
    - cd /Users/rija/MSE1003H_RijaAnsari/Assignment_2
    - git add . 
- Move back to the main repo
    - cd .. 
    - git commit -m "Assignment 2 structure update"
    - git pull origin main
    - git push origin main

### Import data

In [ ]:
pip install ternary

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ternary
import plotly.express as px
import colorsys

In [ ]:
cwd = os.getcwd()
print("Current working directory:", cwd)
#open csv file
input = pd.read_csv("colors3.csv")
output = pd.read_csv("color_results.csv")

In [ ]:
volumes = input.copy()
volumes

In [ ]:
fig = px.scatter_ternary(volumes, a="R", b="Y", c="B", color=volumes["R"] / (volumes["R"] + volumes["Y"] + volumes["B"]), color_continuous_scale="viridis")
fig.update_layout(title="Ternary Plot of Color Ratios")
fig.show()

In [ ]:
#ternary plot 
fig = px.scatter_ternary(
    volumes, 
    a="R", 
    b="Y", 
    c="B",
)

fig.update_traces(marker=dict(color='black', size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()

- talk about colour ratios + robot volume conditions
The colour ratios chosen for the 26 data
- talk about strategy of covering the space
- talk about ordering ie highest yellow concentration first because of impact etc


In [ ]:
results = output.copy()
results

In [ ]:
#find the row where yellow signal is 280
yellow_signal = results[results['Yellow'] == 280].index[0]
red_signal = results[results['Red'] == 280].index[0]
blue_signal = results[results['Blue'] == 280].index[0]

red_signal, yellow_signal, blue_signal

In [ ]:
#create a new dataframe with only channel values from last 8 columns
channels = ['ch410', 'ch440', 'ch470', 'ch510', 'ch550', 'ch583', 'ch620', 'ch670']
results_ch = results[channels]

results_ch

Here we are going to compare our max red, yellow and blue values with our middle point in the ternary diagram to see how the sensor responds to an increase in those values. 

This helps us understand how the wavelengths intensity changes relative to our center point.

In [ ]:
yellow = results_ch.iloc[yellow_signal] - results_ch.iloc[25]
yellow

In [ ]:
red = results_ch.iloc[red_signal] - results_ch.iloc[25]
red

In [ ]:
blue = results_ch.iloc[blue_signal] - results_ch.iloc[25]
blue

In [ ]:
results_relative_center = results_ch - results_ch.iloc[25]
results_relative_center

In [ ]:
results_relative_y = results_ch - results_ch.iloc[red_signal] - results_ch.iloc[blue_signal]
results_relative_y['total_intensity'] = results_relative_y.sum(axis=1)
results_relative_y


In [ ]:
results_relative_yellow = results_ch.iloc[yellow_signal] - results_ch
results_relative_yellow['total_intensity'] = results_relative_yellow.sum(axis=1)
results_relative_yellow

In [ ]:
results_relative_red = results_ch.iloc[red_signal] - results_ch
results_relative_red

In [ ]:
results_relative_blue = results_ch.iloc[blue_signal] - results_ch
results_relative_blue

In [ ]:
"""
# 1. Setup sample data (8 channels of Transmittance % values)
channels = ['ch410', 'ch440', 'ch470', 'ch510', 'ch550', 'ch583', 'ch620', 'ch670']
# Let's assume df_trans contains your transmittance percentages (0-100)
# Example data:
data = {ch: [95.0, 80.0, 50.0] for ch in channels} 
df_trans = pd.DataFrame(data)"""

# 2. Define Constants (Adjust epsilon values based on your specific chemical/dye)
# For this example, we'll assume a path length (b) of 1
b = 1 
# Epsilon values are wavelength-specific. 
# You should replace these 1.0 values with your actual coefficients.
epsilons = {ch: 1.0 for ch in channels} 

# 3. Create the Absorbance Dataframe (A = -log10(T_decimal))
# We divide by 100 to convert percentage transmittance to decimal
df_abs = -np.log10(results_ch / 100)

# 4. Create the Concentration Dataframe (C = A / (e * b))
"""df_conc = pd.DataFrame()

for ch in channels:
    # Applying A = ebc rearranged to C = A / (e * b)
    df_conc[f'{ch}_conc'] = df_abs[ch] / (epsilons[ch] * b)
"""
# 5. Resulting Dataframe
print("Absorbance Values:")
df_abs

#print("\nConcentration Values:")
#print(df_conc.head())

In [ ]:
df_abs['ro_raw'] = df_abs['ch620'] + df_abs['ch670']

# Yellow: Green-Yellow to Amber wavelengths
df_abs['yo_raw'] = df_abs['ch510'] + df_abs['ch550'] + df_abs['ch583']

# Blue: Violet to Blue-Cyan wavelengths
df_abs['bo_raw'] = df_abs['ch410'] + df_abs['ch440'] + df_abs['ch470']

# 3. Calculate the total intensity for normalization
df_abs['total_intensity'] = df_abs['ro_raw'] + df_abs['yo_raw'] + df_abs['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
df_abs['ro'] = (df_abs['ro_raw'] / df_abs['total_intensity']) * 300
df_abs['yo'] = (df_abs['yo_raw'] / df_abs['total_intensity']) * 300
df_abs['bo'] = (df_abs['bo_raw'] / df_abs['total_intensity']) * 300

# 5. Clean up: keep only the outputs you want
results_out = df_abs[['ro', 'yo', 'bo']].copy()

print(results_out)

In [ ]:
results['ro_raw'] = results['ch620'] + results['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results['yo_raw'] = results['ch510'] + results['ch550'] + results['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results['bo_raw'] = results['ch410'] + results['ch440'] + results['ch470']

# 3. Calculate the total intensity for normalization
results['total_intensity'] = results['ro_raw'] + results['yo_raw'] + results['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results['ro'] = (results['ro_raw'] / results['total_intensity']) * 300
results['yo'] = (results['yo_raw'] / results['total_intensity']) * 300
results['bo'] = (results['bo_raw'] / results['total_intensity']) * 300

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_out = results[['ro', 'yo', 'bo']].copy()

print(results_out)

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_out, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")
fig.show()

In [ ]:
results_relative_center['ro_raw'] = results_relative_center['ch620'] + results_relative_center['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results_relative_center['yo_raw'] = results_relative_center['ch510'] + results_relative_center['ch550'] + results_relative_center['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results_relative_center['bo_raw'] = results_relative_center['ch410'] + results_relative_center['ch440'] + results_relative_center['ch470']

# 3. Calculate the total intensity for normalization
results_relative_center['total_intensity'] = results_relative_center['ro_raw'] + results_relative_center['yo_raw'] + results_relative_center['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results_relative_center['ro'] = (results_relative_center['ro_raw'] / results_relative_center['total_intensity']) * 300
results_relative_center['yo'] = (results_relative_center['yo_raw'] / results_relative_center['total_intensity']) * 300
results_relative_center['bo'] = (results_relative_center['bo_raw'] / results_relative_center['total_intensity']) * 300

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_center = results_relative_center[['ro', 'yo', 'bo']].copy()

print(results_center)

In [ ]:
results_center.iloc[25] = [1, 1, 1]

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_center, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")
fig.show()

In [ ]:
results_relative_red['ro_raw'] = results_relative_red['ch620'] + results_relative_red['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results_relative_yellow['yo_raw'] = results_relative_yellow['ch510'] + results_relative_yellow['ch550'] + results_relative_yellow['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results_relative_blue['bo_raw'] = results_relative_blue['ch410'] + results_relative_blue['ch440'] + results_relative_blue['ch470']

# 3. Calculate the total intensity for normalization
results_relative = pd.DataFrame()  # Create an empty DataFrame to store results
results_relative['total_intensity'] = results_relative_red['ro_raw'] + results_relative_red['yo_raw'] + results_relative_red['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results_relative['ro'] = (results_relative_red['ro_raw'] / results_relative['total_intensity']) * 300
results_relative['yo'] = (results_relative_yellow['yo_raw'] / results_relative['total_intensity']) * 300
results_relative['bo'] = (results_relative_blue['bo_raw'] / results_relative['total_intensity']) * 300

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_relative = results_relative[['ro', 'yo', 'bo']].copy()

print(results_relative)

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_relative, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")
fig.show()

In [ ]:
def convert_spectral_to_ryb(data):
    # Sum the raw channel intensities
    r_raw = data['ch620'] + data['ch670']
    y_raw = data['ch510'] + data['ch550'] + data['ch583']
    b_raw = data['ch410'] + data['ch440'] + data['ch470']
    
    total = r_raw + y_raw + b_raw
    
    # Avoid division by zero
    if total == 0:
        return 0, 0, 0
    
    # Convert to percentages for ternary plot
    r_perc = (r_raw / total) * 100
    y_perc = (y_raw / total) * 100
    b_perc = (b_raw / total) * 100
    
    return r_perc, y_perc, b_perc

# Example Usage:
sensor_readings = {
    'ch410': 10, 'ch440': 20, 'ch470': 15, # Blueish
    'ch510': 5,  'ch550': 2,  'ch583': 1,  # Yellowish
    'ch620': 50, 'ch670': 80               # Reddish
}

r, y, b = convert_spectral_to_ryb(sensor_readings)
print(f"Red: {r:.2f}%, Yellow: {y:.2f}%, Blue: {b:.2f}%")

In [ ]:
results['ro_raw'] = results['ch620'] + results['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results['yo_raw'] = results['ch510'] + results['ch550'] + results['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results['bo_raw'] = results['ch410'] + results['ch440'] + results['ch470']

# 3. Calculate the total intensity for normalization
results['total_intensity'] = results['ro_raw'] + results['yo_raw'] + results['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results['ro'] = (results['ro_raw'] / results['total_intensity']) * 300
results['yo'] = (results['yo_raw'] / results['total_intensity']) * 300
results['bo'] = (results['bo_raw'] / results['total_intensity']) * 300

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_out = results[['ro', 'yo', 'bo']].copy()

print(results_out)